# Migration of External Storage from DataJoint 0.11.* to 0.12.*
DataJoint 0.12 improves the efficiency of blob storage and expands its capabilities. Unfortunately, this breaks backward compatibility for external storage used in previous versions. This notebook describes the migration procedure from a 0.11 external store.

First, let's emulate an legacy table with external storage. You do not need to perform these steps if you are migrating an existing database. I `git`-cloned `datajoint-python` as a subfolder in the datajoint folder, checked out the legacy version `v0.11.1`, and renamed the subfolder into `dj011`. This allows me to import the legacy version of datajoint while keeping the current version available as well:

In [1]:
import dj011 as dj   # legacy version of datajoint

In [2]:
dj.__version__

'0.11.1'

In [3]:
dj.config['database.password'] = 'datajoint'
dj.config['database.user'] = 'datajoint'
dj.config['database.host'] = 'localhost'

In [4]:
schema = dj.schema('djtest_blobs')
schema.drop()
schema = dj.schema('djtest_blobs')

Connecting datajoint@localhost:3306
Proceed to delete entire schema `djtest_blobs`? [yes, No]: yes


In [5]:
# Configure stores
import os 

dj.config['external'] = dict(
    protocol='s3',
    endpoint="localhost:9000",
    bucket='migrate-test',
    location='store',
    access_key="datajoint",
    secret_key="datajoint")

dj.config['external-shared'] = dict(
    protocol='s3',
    endpoint="localhost:9000",
    bucket='migrate-test',
    location='maps',
    access_key="datajoint",
    secret_key="datajoint")

dj.config['external-local'] = dict(
    protocol='file',
    location=os.path.expanduser('~/temp/migrate-test'),
    access_key="datajoint",
    secret_key="datajoint")


dj.config['cache'] = os.path.expanduser('~/temp/dj-cache')

Now let's define the legacy-style table with external blobs:

In [6]:
@schema
class A(dj.Manual):
    definition = """
    id : int 
    ---
    blob_external : external    # uses S3
    blob_share : external-shared   # uses S3
    """

@schema
class B(dj.Manual):
    definition = """
    id : int 
    ---
    blob_local : external-local    # uses files
    blob_share : external-shared   # uses S3
    """

In [7]:
import numpy as np

In [8]:
A.insert((
    (0, np.random.randn(2,3,4), np.random.randn(3)),
    (1, np.array([1,2,3]), np.array([1,2]))
))

B.insert((
    (0, np.random.randn(2,3,4), np.random.randn(3)),
    (1, np.array([1,2,3]), np.array([1,2]))
))

# Upgrade legacy blobs
(restart kernel)

In [1]:
import datajoint as dj

In [2]:
dj.__version__

'0.12.dev4'

In [3]:
dj.config['database.password'] = 'datajoint'
dj.config['database.user'] = 'datajoint'
dj.config['database.host'] = 'localhost'

In [4]:
schema = dj.schema('djtest_blobs')
query = schema.connection.query

Connecting datajoint@localhost:3306


In [5]:
# Configure stores
import os 

default_store = 'external'  # naming the unnamed external store

dj.config['stores'] = {
    
    default_store: dict(
        protocol='s3',
        endpoint="localhost:9000",
        bucket='migrate-test',
        location='store',
        access_key="datajoint",
        secret_key="datajoint"),
    
    'shared': dict(
        protocol='s3',
        endpoint="localhost:9000",
        bucket='migrate-test',
        location='maps',
        access_key="datajoint",
        secret_key="datajoint"),
    
    'local': dict(
        protocol='file',
        location=os.path.expanduser('~/temp/migrate-test'))
}

dj.config['cache'] = os.path.expanduser('~/temp/dj-cache')

In [6]:
@schema
class C(dj.Manual):
    definition = """
    id : int
    ---
    blo : blob@shared   # just a check
    """
    

In [7]:
C.describe();

id                   : int                          
---
blo                  : blob@shared                  # just a check
INDEX (blo)



In [8]:
schema.spawn_missing_classes()

In [9]:
A()

DataJointError: Legacy datatype `external`.

DataJointError: Legacy datatype `external`.

In [16]:
LEGACY_HASH_SIZE = 43

legacy_external = dj.FreeTable(
    schema.connection,
    '`{db}`.`~external`'.format(db=schema.database))

In [17]:
legacy_external

hash the hash of stored object + store name,size size of object in bytes,timestamp automatic timestamp
AbOdLZWSNmvYrfESJci85yLPplWIUM9E7UmyzHg0ApM,237,2019-07-19 13:22:54
BEq9eh9LlqkPOKS8PwbqsOX3PTom0MhLmqvlN43yfrsshared,53,2019-07-19 13:22:54
FoRROa2LWM6_wx0RIQ0J-LVvgm256cqDQfJa066HoTEshared,37,2019-07-19 13:22:54
l3MDivFfPe1GV74Fdyky4YSLVKq3Y4x_U7mtzAReSaUlocal,237,2019-07-19 13:22:54
U7u2I13bb6Zx2bC7yn_J8yCYos6fDebQBJhaUf4ho2Eshared,53,2019-07-19 13:22:54
_Fhi2GUBB0fgxcSP2q-isgncIUTdgGK7ivHiySAU_94,40,2019-07-19 13:22:54
_Fhi2GUBB0fgxcSP2q-isgncIUTdgGK7ivHiySAU_94local,40,2019-07-19 13:22:54


In [18]:
# get referencing tables
refs = query("""
SELECT concat('`', table_schema, '`.`', table_name, '`') as referencing_table, column_name, constraint_name
FROM information_schema.key_column_usage
WHERE referenced_table_name="{tab}" and referenced_table_schema="{db}"
""".format(tab=legacy_external.table_name, db=legacy_external.database), as_dict=True).fetchall()

In [19]:
import re

for ref in refs:
    # get comment
    column = query(
        'SHOW FULL COLUMNS FROM {referencing_table}'
        'WHERE Field="{column_name}"'.format(**ref), as_dict=True).fetchone()

    store, comment = re.match(
        r':external(-(?P<store>.+))?:(?P<comment>.*)', 
        column['Comment']).group('store', 'comment')

    # get all the hashes from the reference
    hashes = {x[0] for x in query(
        'SELECT `{column_name}` FROM {referencing_table}'.format(**ref))}

    # sanity check make sure that store suffixes match
    if store is None:
        assert all(len(_) == LEGACY_HASH_SIZE for _ in hashes)
    else:
        assert all(_[LEGACY_HASH_SIZE:] == store for _ in hashes)

    # create new-style external table
    ext = schema.external[store or default_store]

    # add the new-style reference field
    temp_suffix = 'tempsub'

    try:
        query("""ALTER TABLE {referencing_table} 
        ADD COLUMN `{column_name}_{temp_suffix}` {type} DEFAULT NULL
        COMMENT ":blob@{store}:{comment}"
        """.format(type=dj.declare.UUID_DATA_TYPE, 
                   temp_suffix=temp_suffix, 
                   store=(store or default_store), comment=comment, **ref))
    except:
        print('Column already added')
        pass


    # Copy references into the new external table
    # No Windows! Backslashes will cause problems

    contents_hash_function = {
        'file': lambda ext, relative_path: dj.hash.uuid_from_file(os.path.join(ext.spec['location'], relative_path)),
        's3': lambda ext, relative_path: dj.hash.uuid_from_buffer(ext.s3.get(relative_path))
    }

    for _hash, size in zip(*legacy_external.fetch('hash', 'size')):
        if _hash in hashes:
            relative_path = os.path.join(schema.database, _hash)
            uuid = dj.hash.uuid_from_buffer(init_string=relative_path)
            ext.insert1(dict(
                filepath=relative_path,
                size=size,
                contents_hash=contents_hash_function[ext.spec['protocol']](ext, relative_path),
                hash=uuid
            ), skip_duplicates=True)

            query('UPDATE {referencing_table} '
                  'SET `{column_name}_{temp_suffix}`=%s '
                  'WHERE `{column_name}` = "{_hash}"'
                  .format(_hash=_hash, temp_suffix=temp_suffix, **ref), uuid.bytes)

    # check that all have been copied
    check = query('SELECT * FROM {referencing_table} '
                  'WHERE `{column_name}` IS NOT NULL'
                  '  AND `{column_name}_{temp_suffix}` IS NULL'
                  .format(temp_suffix=temp_suffix, **ref)).fetchall()

    assert len(check) == 0, 'Some hashes havent been migrated'

    # drop old foreign key, rename, and create new foreign key
    query("""
        ALTER TABLE {referencing_table}
        DROP FOREIGN KEY `{constraint_name}`,
        DROP COLUMN `{column_name}`,
        CHANGE COLUMN `{column_name}_{temp_suffix}` `{column_name}` {type} DEFAULT NULL
             COMMENT ":blob@{store}:{comment}",
        ADD FOREIGN KEY (`{column_name}`) REFERENCES {ext_table_name} (`hash`)
        """.format(temp_suffix=temp_suffix, 
                   ext_table_name=ext.full_table_name, 
                  type=dj.declare.UUID_DATA_TYPE, 
                  store=(store or default_store), comment=comment, **ref))

In [20]:
legacy_external

hash the hash of stored object + store name,size size of object in bytes,timestamp automatic timestamp
AbOdLZWSNmvYrfESJci85yLPplWIUM9E7UmyzHg0ApM,237,2019-07-19 13:22:54
BEq9eh9LlqkPOKS8PwbqsOX3PTom0MhLmqvlN43yfrsshared,53,2019-07-19 13:22:54
FoRROa2LWM6_wx0RIQ0J-LVvgm256cqDQfJa066HoTEshared,37,2019-07-19 13:22:54
l3MDivFfPe1GV74Fdyky4YSLVKq3Y4x_U7mtzAReSaUlocal,237,2019-07-19 13:22:54
U7u2I13bb6Zx2bC7yn_J8yCYos6fDebQBJhaUf4ho2Eshared,53,2019-07-19 13:22:54
_Fhi2GUBB0fgxcSP2q-isgncIUTdgGK7ivHiySAU_94,40,2019-07-19 13:22:54
_Fhi2GUBB0fgxcSP2q-isgncIUTdgGK7ivHiySAU_94local,40,2019-07-19 13:22:54


In [23]:
# Drop the old external table but make sure it's no longer referenced
# get referencing tables
refs = query("""
SELECT concat('`', table_schema, '`.`', table_name, '`') as referencing_table, column_name, constraint_name
FROM information_schema.key_column_usage
WHERE referenced_table_name="{tab}" and referenced_table_schema="{db}"
""".format(tab=legacy_external.table_name, db=legacy_external.database), as_dict=True).fetchall()

assert not refs, 'Some references still exist'

In [26]:
# drop old external table
legacy_external.drop_quick()

In [36]:
schema.external['external']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,filepath relative filepath used in the filepath datatype,contents_hash used for the filepath datatype,timestamp automatic timestamp
83186db4-c969-0298-3ec8-1da403c715d5,40,djtest_blobs/_Fhi2GUBB0fgxcSP2q-isgncIUTdgGK7ivHiySAU_94,069fe219-23b4-62e5-fbfe-da572fc47f7d,2019-07-19 13:34:30
b052bba1-018d-ec41-3c35-33bb7df4cb90,237,None,None,2019-07-19 13:41:29
d4b9b701-b577-d088-e6f7-135f060b9cfb,237,djtest_blobs/AbOdLZWSNmvYrfESJci85yLPplWIUM9E7UmyzHg0ApM,69391462-c54e-1f19-2f78-7f3f2eccb9b9,2019-07-19 13:34:30
ea5894dc-3a46-5ebe-4bbd-e80ebf08b161,53,None,None,2019-07-19 13:41:29


In [35]:
import datajoint as dj

dj.config['database.password'] = 'datajoint'
dj.config['database.user'] = 'datajoint'
dj.config['database.host'] = 'localhost'

schema = dj.schema('djtest_blobs')
query = schema.connection.query

In [ ]:
schema.spawn_missing_classes()

In [ ]:
dj.ERD(schema)

In [32]:
import numpy as np

In [33]:
A.insert((
    (2, np.random.randn(2,3,4), np.random.randn(3)),
    (3, np.array([1,2,3]), np.array([1,2]))
))

In [ ]:
A.fetch('blob_share')

In [ ]:
import os

In [ ]:
os.path.join('one', 'two', 'three\\four')